In [4]:
# Linear Binary Classification
# Given a set of N points in the D-dimensional space and their labels of boolean values, find a hyperplane that separates the true and false class. Return a classifier that takes in a point vector and returns probility.

import numpy as np

def binary_classifier_from(X, Y):
    '''
        X, a float np.array of shape [N, D]
        Y, a boolean array of labels, shape [N]
    '''
    epsilon = 1e-5

    D = X.shape[1]

    np.random.seed(42)
    W = np.random.randn(D)  # D,
    b = 0  # 1

    lr = 0.001

    def sigmoid(x):
        return (1 + np.exp(-x)) ** -1
    def d_sigmoid(x):
        # -1 * (1 + np.exp(-x)) ** -2 * np.exp(-x) * -1
        # return sigmoid(x) ** 2 * np.exp(-x)  
        return  sigmoid(x) * (1 -  sigmoid(x))
    
    m = X.mean(0)
    std = X.std()

    def forward(vectors):
        standardized_v = (vectors - m) / (std + epsilon)
        logit = standardized_v @ W + b  # N
        prob = sigmoid(logit)  # N
        return prob

    for i in range(200):
        # Forward
        prob = forward(X)  # N
        labels = Y * 1.0
        loss = - labels * np.log(prob + epsilon) - (1 - labels) * np.log(1 - prob + epsilon)

        # Backward
        standardized_x = (X - m) / (std + epsilon)
        # d_prob = - labels / (prob + epsilon) + (1 - labels) / (1 - prob + epsilon)  # N
        # d_logit = prob * (1 - prob) * d_prob # N
        d_logit = prob - labels
        d_W = standardized_x.T @ d_logit  # D
        d_b = d_logit.sum()

        W -= lr * d_W
        b -= lr * d_b

        print(f"loss.mean={loss.mean()}")
        # break

    return forward


# train_data = np.array([
#     [0, 1], [1, 2], [3, 4], [1, 1], [2, 3],
# ], dtype=float)
# train_labels = np.array([True, True, True, False, True])

# train_data = np.array([
#     [0, 1], [1, 2],
# ], dtype=float)
# train_labels = np.array([True, False])

train_data = np.concatenate(
    [
        np.random.randn(100, 10) + np.array([50, 20] + [0] * 8, dtype=float),
        np.random.randn(100, 10) + np.array([20, 0] + [0] * 8, dtype=float),
    ], axis=0)
train_labels = np.array([True] * 100 + [False] * 100)

model = binary_classifier_from(train_data, train_labels)
print(model(train_data))

loss.mean=0.4802024482630155
loss.mean=0.41946053767487257
loss.mean=0.3703146053091112
loss.mean=0.33010736457692164
loss.mean=0.29683976773924464
loss.mean=0.26901033716205247
loss.mean=0.2454868325817521
loss.mean=0.2254092792376342
loss.mean=0.20811861929372028
loss.mean=0.19310480097935254
loss.mean=0.17996910067031546
loss.mean=0.16839669998151433
loss.mean=0.15813661069196466
loss.mean=0.14898686740825665
loss.mean=0.14078351259024305
loss.mean=0.13339232917103505
loss.mean=0.12670257900271087
loss.mean=0.12062221776301556
loss.mean=0.11507420600939572
loss.mean=0.10999364107684464
loss.mean=0.10532550892569742
loss.mean=0.10102290813378893
loss.mean=0.09704563638740246
loss.mean=0.09335905746919863
loss.mean=0.08993318692567762
loss.mean=0.08674194945110557
loss.mean=0.08376257204196605
loss.mean=0.08097508520950375
loss.mean=0.07836191073682626
loss.mean=0.07590751916769006
loss.mean=0.07359814380342901
loss.mean=0.07142154074360976
loss.mean=0.06936678664065027
loss.mean=0.06

With 
        d_prob = - labels / (prob + epsilon) + (1 - labels) / (1 - prob + epsilon)  # N
        d_logit = prob * (1 - prob) * d_prob # N
in the backprob, it's very same:

loss.mean=0.47989471852639026
loss.mean=0.4190224805366235
loss.mean=0.3698198680071503
loss.mean=0.32959864629758917
loss.mean=0.2963408932624755
loss.mean=0.26853386425829784
loss.mean=0.24503870297758878
loss.mean=0.22499161669233395
loss.mean=0.20773139806909896
loss.mean=0.1927468346361424
loss.mean=0.17963862408216955
loss.mean=0.16809170825330783
loss.mean=0.1578550528428587
loss.mean=0.14872675337547514